In [1]:
import pandas
df = pandas.read_csv("goal_prediction.csv")

In [2]:
print(df.shape)

(5592, 41)


In [3]:
print(df.columns)

Index(['match_id', 'scored', 'fpl_minutes', 'fpl_goals', 'fpl_assits',
       'fpl_cs', 'fpl_points', 'fpl_bonus', 'fpl_cprice', 'player_rating',
       'touches', 'saves', 'total_pass', 'aerial_won', 'formation_place',
       'accurate_pass', 'total_tackle', 'aerial_lost', 'fouls', 'yellow_card',
       'total_scoring_att', 'man_of_the_match', 'goals', 'won_contest',
       'blocked_scoring_att', 'goal_assist', 'good_high_claim',
       'second_yellow', 'red_card', 'error_lead_to_goal', 'last_man_tackle',
       'six_yard_block', 'post_scoring_att', 'att_pen_target', 'penalty_save',
       'penalty_conceded', 'clearance_off_line', 'att_pen_goal',
       'att_pen_miss', 'own_goals', 'att_pen_post'],
      dtype='object')


In [9]:
home_win = []
away_win = []
draw = []

for i in df['winner']:
    if i == 0:
        draw.append(1)
        home_win.append(0)
        away_win.append(0)
    elif i== -1:
        draw.append(0)
        home_win.append(0)
        away_win.append(1)
    else:
        draw.append(0)
        home_win.append(1)
        away_win.append(0)
        

In [10]:
df['home_wins'] = home_win
df['away_wins'] = away_win
df['draws'] = draw

In [11]:
print(df.shape)

(1520, 971)


In [19]:
X = df.iloc[:,8:968]

In [20]:
X.shape

(1520, 960)

In [22]:
Y = df.iloc[:,968:]
Y.shape

(1520, 3)

In [1]:
#Torch Classes
#General Data processing, Linear Algebra and Plotting  classes

from __future__ import print_function, division
import os
import sys
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import struct
import math
from torch.utils import data
# Ignore warnings
import warnings
import csv
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim

from sklearn.metrics import confusion_matrix

cuda = torch.device('cuda')

In [2]:
class Football(nn.Module):
    def __init__(self):
        super(Football,self).__init__()
        
        
        self.fc1 = nn.Linear(39,20)
        #self.fc2 = nn.Linear(50,20)
        self.fc3 = nn.Linear(20,5)
        #self.fc4 = nn.Linear(100,50)
        #self.fc5 = nn.Linear(50,20)
        self.softmax = nn.Linear(5,1)
        
    def forward(self,x,**kwargs):
        
        #print(x)
        x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        #x = F.sigmoid(self.fc4(x))
        #x = F.sigmoid(self.fc5(x))
        x = F.sigmoid(self.softmax(x))
        # print(x)
   
        return x
        

In [28]:
import sys
def training_1(net, X,Y, criterion,optimizer,epoch=5):
    #palm_joints_list = [0:13]
    #phalanges_joints_list = []

    total_loss = 0

    for e in range(epoch):
        running_loss = 0.0
        print("Starting Epoch %d..."%(e+1))
        for i,(inputs,labels) in enumerate(zip(X,Y),0):
            
            inputs = torch.from_numpy(inputs)
            inputs = inputs.type(torch.FloatTensor)
        
            optimizer.zero_grad()
            y_pred = net(inputs)
            labels = torch.from_numpy(np.array(labels))
            labels.requires_grad = False
            labels = labels.type(torch.FloatTensor)
            loss = criterion(y_pred,labels)
            
            if y_pred == 1:
                loss = loss*2 
            loss.backward()
            optimizer.step()
        
        #printing stats 
            sys.stdout.write('\r [epoch: %d, iteration: %5d] loss: %.3f ' % (e+1,i+1,running_loss/200))
            sys.stdout.flush()
        
            running_loss += loss.item()
            total_loss += loss.item()**0.5
            
        
        torch.save(net,"models/football_1_%d.pth"%(e+1))
        torch.save(optimizer,"optim.pth")
        
        total_loss = 0
        print("\nEpoch %d complete. Model after epoch %d successfully saved as models/football_net_1_%d.pth..."%(e+1,e+1,e+1))
        
    print('Finished Training...')

In [29]:
dff = pd.read_csv("goal_prediction.csv")
#dff = dff.iloc[:,1:]

In [30]:
criterion = nn.BCELoss()
football_net = Football()
print(football_net)
optimizer = optim.Adam(football_net.parameters(),weight_decay = float(0.0001),lr= float(0.001))

Football(
  (fc1): Linear(in_features=39, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=5, bias=True)
  (softmax): Linear(in_features=5, out_features=1, bias=True)
)


In [31]:
msk = np.random.rand(len(dff)) < 0.9

train = dff[msk]

test = dff[~msk]

In [ ]:
training_1(football_net,dff.iloc[:,2:].values,dff.iloc[:,1].values,criterion,optimizer,epoch = 10)

Starting Epoch 1...
 [epoch: 1, iteration:   644] loss: 11.152 

In [114]:
net = torch.load("models/football_1_10.pth")

In [115]:
error = 0
#total_sum = 0
max,min = -100,100
predicts = []
for row in dff.values:
    x = row[2:]
    x = torch.from_numpy(np.array(x))
    x = x.type(torch.FloatTensor)
    y = row[1]
    pred = net(x).tolist()[0]
    
    if max < pred:
        max = pred
    if min > pred:
        min = pred
    
    '''if pred > 0.5:
        pred=1
    else :
        pred=0'''
    #print(y,pred)
    pred = round(pred)
    #total_sum += pred 
    predicts.append(pred)
    error += (abs(y-pred))

In [116]:
print(sum(predicts))

3105


In [117]:
sum(dff.iloc[:,1])

2796.0

In [118]:
print(1-sum(abs(predicts - dff.iloc[:,1]))/len(dff))

0.7533977110157368


In [119]:
confmat = confusion_matrix(y_true = dff.iloc[:,1].tolist(),y_pred = predicts)

In [120]:
print(confmat)

[[1952  844]
 [ 535 2261]]
